In [1]:
cd ..

C:\Users\johan\github\designresolver


In [2]:
from execution import Component

In [3]:
from autograd import jacobian, grad
import autograd.numpy as anp
import numpy as np

In [74]:
fx = lambda x: x[0]+x[1]+x[2]
g = grad(fx)

In [75]:
g(x)

array([1., 1., 1.])

In [87]:
vector_fun = lambda x: anp.array([x[0]+x[1]+x[2], x[0]+x[2]])

x = np.random.randn(3)
jac = jacobian(vector_fun)(x)

In [88]:
jac

array([[1., 1., 1.],
       [1., 0., 1.]])

# Components
Different examples of using components

## Plain

In [4]:
vector_fun = lambda *x: anp.array([x[0]+x[1]+x[2], x[0]+x[2]])

In [9]:
# Inputs should not be same as outputs!
inputs = (1,2,3)
outputs = (4,5)
c1 = Component(vector_fun, inputs, outputs, 1)

In [6]:
c1.jacobian(np.array([2.,4.,3.]))

array([[1., 1., 1.],
       [1., 0., 1.]])

In [7]:
c1.evaldict({1: 2, 2:4, 3:3})

array([9, 5])

## Inputs and outputs with units

In [24]:
inputs = (1,2,3) #The order is critical
# we describe inunits as a dictionary to decouple from inputs
inunits = {1:'kg', 2: 'g', 3:'m/s^2'}
outputs = (4,5)
outunits = {4:'N', 5:'g'}

In [37]:
from compute import ureg
from unitutils import get_unit_multiplier

In [34]:
def expression_conversion_unit(expr_unit, tounit=None):
    unit = tounit if tounit else ureg('')
    if tounit:
        assert(unit.dimensionality == expr_unit.dimensionality)
        conversion_unit = unit
    else: # tounit was not given
        if not hasattr(expr_unit, 'units'):
            conversion_unit = ureg('')
        else:
            # if we evaluate an expression and get some crazy unit 
            # we bring it back to it's base dimensionality
            conversion_unit = ureg.Quantity(1, 
                expr_unit.to_base_units().units)
    return conversion_unit

In [127]:
def unit_conversion_factors(outunitpairs, inunits):
    convert = np.array([get_unit_multiplier(inp) for 
            inp in inunits])
    factors = []
    for outunit, tounit in outunitpairs:
        conversion_unit = expression_conversion_unit(outunit, tounit)
        factor = get_unit_multiplier(conversion_unit)
        factors.append(factor)
    return convert, factors

In [128]:
expression_conversion_unit(ureg.Quantity(1, 'km'))

1 <Unit('meter')>

In [129]:
unit_conversion_factors([(ureg.Quantity(1, 'g*m/s^2'), ureg.Quantity(1, 'mN'))], [ureg.Quantity(1, 'km'), ureg.Quantity(1, 'kg')])

(array([1000.,    1.]), [0.001])

In [60]:
from unitutils import MockFloat
import sympy as sp

In [134]:
def get_unit(fx, inputunits):
    args = (ureg.Quantity(MockFloat(1), inputunit) for inputunit in inputunits)
    dim = fx(*args)
    dims = listify(dim)
    # need this case if output is a float, which can happen when we have powers, e.g. 10^x:
    dims = [dim if isinstance(dim, ureg.Quantity) else ureg('') for dim in dims]
    return dims

In [55]:
get_unit(lambda *x: (x[0]+x[1])*x[2], ('g', 'kg', 'm/s^2'))

1.0 <Unit('gram * meter / second ** 2')>

In [133]:
def listify(out):
    return out if isinstance(out, list) else [out]

In [185]:
def flatten_list(ls):
    return ls if np.isscalar(ls)==1 else list(ls)

In [186]:
def executable_with_conversion(convert, factors, fx):
    def scaled_fx(*args):
        return flatten_list(np.array(fx(*(convert*np.array(args).flatten())))/factor)
    return scaled_fx

In [62]:
from compute import Var

In [63]:
a = Var('a', 1, 'g')
b = Var('b', 2, 'kg')
c = Var('c', 3, 'm/s^2')

In [159]:
expr1 = (a+b)*c
expr2 = a-b

In [190]:
inputs = (a,b,c)
inpunitsflat = tuple(inpvar.varunit for inpvar in inputs)
fx = sp.lambdify(inputs, [expr1, expr2], anp) #anp_math
#outputs = (None,)
outunitsflat = (ureg.Quantity('N'),ureg.Quantity('g'))

In [191]:
def fx_with_units(fx, inunitsflat, outunitsflat):
    expr_units = get_unit(fx, inunitsflat)
    outunitpairs = ((outunit, outunitsflat[idx]) for idx, outunit in enumerate(expr_units))
    convert, factors = unit_conversion_factors(outunitpairs, inunitsflat)
    fx_scaled = executable_with_conversion(convert, factors, fx)
    return fx_scaled

In [192]:
fx_scaled = fx_with_units(fx, inpunitsflat, outunitsflat)

In [193]:
fx_scaled(1,2,3)

[6.003, -1.999]